In [1]:
!pip install ctgan
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [2]:
from ctgan import CTGAN

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import time

# record start time
start = time.time()
le = LabelEncoder()

df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/insurance.csv')

#transform data into numeric value
df1=df[df['labels']==0]
df=df[df['labels']==1]


#df=df.drop(['labels'],axis=1)
#df1=df1.drop(['labels'],axis=1)


batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(df)

end = time.time()



print("The time of execution of preprocess:",
      (end-start), "s")

Gen. (-1.25) | Discrim. (0.09): 100%|██████████| 100/100 [01:23<00:00,  1.20it/s]

The time of execution of preprocess: 111.89674401283264 s


In [3]:
n_generated_data = 25974
generated_df = model.sample(n_generated_data)

#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
from sklearn.ensemble import RandomForestClassifier
import math
import time

startu=time.time()




X=data.drop(['labels'],axis=1)
y=data["labels"]


#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)
cv = KFold(n_splits=5, random_state=1, shuffle=True)

endu=time.time()
print("The time of execution of preprocess undersampling:",
      (endu-startu), "s")


The time of execution of preprocess undersampling: 0.00939035415649414 s


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.926010 using {'class_weight': {0: 100, 1: 100}}
0.924936 (0.002007) with: {'class_weight': {0: 100, 1: 10}}
0.925170 (0.002032) with: {'class_weight': {0: 100, 1: 20}}
0.925392 (0.001623) with: {'class_weight': {0: 100, 1: 30}}
0.925264 (0.001658) with: {'class_weight': {0: 100, 1: 40}}
0.925548 (0.001832) with: {'class_weight': {0: 100, 1: 50}}
0.925431 (0.001836) with: {'class_weight': {0: 100, 1: 60}}
0.925309 (0.001681) with: {'class_weight': {0: 100, 1: 70}}
0.925793 (0.001922) with: {'class_weight': {0: 100, 1: 80}}
0.925910 (0.001827) with: {'class_weight': {0: 100, 1: 90}}
0.926010 (0.001842) with: {'class_weight': {0: 100, 1: 100}}


In [7]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.926137 using {'class_weight': {0: 100, 1: 100}}
0.924749 (0.001817) with: {'class_weight': {0: 100, 1: 10}}
0.925350 (0.001850) with: {'class_weight': {0: 100, 1: 20}}
0.925311 (0.001751) with: {'class_weight': {0: 100, 1: 30}}
0.925134 (0.001901) with: {'class_weight': {0: 100, 1: 40}}
0.925512 (0.001754) with: {'class_weight': {0: 100, 1: 50}}
0.925062 (0.002095) with: {'class_weight': {0: 100, 1: 60}}
0.925046 (0.002028) with: {'class_weight': {0: 100, 1: 70}}
0.925725 (0.001786) with: {'class_weight': {0: 100, 1: 80}}
0.925965 (0.001959) with: {'class_weight': {0: 100, 1: 90}}
0.926137 (0.001775) with: {'class_weight': {0: 100, 1: 100}}


In [8]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder


Best: 0.926059 using {'class_weight': {0: 100, 1: 100}}
0.924672 (0.001962) with: {'class_weight': {0: 100, 1: 10}}
0.925255 (0.001587) with: {'class_weight': {0: 100, 1: 20}}
0.925524 (0.001608) with: {'class_weight': {0: 100, 1: 30}}
0.924740 (0.001731) with: {'class_weight': {0: 100, 1: 40}}
0.925339 (0.001445) with: {'class_weight': {0: 100, 1: 50}}
0.925498 (0.001846) with: {'class_weight': {0: 100, 1: 60}}
0.925299 (0.001713) with: {'class_weight': {0: 100, 1: 70}}
0.925532 (0.001608) with: {'class_weight': {0: 100, 1: 80}}
0.925993 (0.001896) with: {'class_weight': {0: 100, 1: 90}}
0.926059 (0.002189) with: {'class_weight': {0: 100, 1: 100}}


In [9]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.911439 using {'class_weight': {0: 100, 1: 100}}
0.896448 (0.002306) with: {'class_weight': {0: 100, 1: 10}}
0.902712 (0.002085) with: {'class_weight': {0: 100, 1: 20}}
0.905721 (0.001966) with: {'class_weight': {0: 100, 1: 30}}
0.907525 (0.001884) with: {'class_weight': {0: 100, 1: 40}}
0.908730 (0.001820) with: {'class_weight': {0: 100, 1: 50}}
0.909589 (0.001770) with: {'class_weight': {0: 100, 1: 60}}
0.910231 (0.001727) with: {'class_weight': {0: 100, 1: 70}}
0.910728 (0.001691) with: {'class_weight': {0: 100, 1: 80}}
0.911123 (0.001658) with: {'class_weight': {0: 100, 1: 90}}
0.911439 (0.001634) with: {'class_weight': {0: 100, 1: 100}}


In [10]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-insurance-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-insurance-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.835181 using {'class_weight': {0: 100, 1: 100}}
0.628874 (0.004695) with: {'class_weight': {0: 100, 1: 10}}
0.709467 (0.003195) with: {'class_weight': {0: 100, 1: 20}}
0.752293 (0.002336) with: {'class_weight': {0: 100, 1: 30}}
0.779751 (0.003120) with: {'class_weight': {0: 100, 1: 40}}
0.798132 (0.002641) with: {'class_weight': {0: 100, 1: 50}}
0.811243 (0.003192) with: {'class_weight': {0: 100, 1: 60}}
0.820849 (0.003102) with: {'class_weight': {0: 100, 1: 70}}
0.827899 (0.002847) with: {'class_weight': {0: 100, 1: 80}}
0.832179 (0.002631) with: {'class_weight': {0: 100, 1: 90}}
0.835181 (0.002426) with: {'class_weight': {0: 100, 1: 100}}


In [11]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-insurance-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-insurance-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.834770 using {'class_weight': {0: 100, 1: 100}}
0.585213 (0.005813) with: {'class_weight': {0: 100, 1: 10}}
0.684541 (0.003906) with: {'class_weight': {0: 100, 1: 20}}
0.736299 (0.002755) with: {'class_weight': {0: 100, 1: 30}}
0.769257 (0.003382) with: {'class_weight': {0: 100, 1: 40}}
0.791217 (0.002799) with: {'class_weight': {0: 100, 1: 50}}
0.806740 (0.003296) with: {'class_weight': {0: 100, 1: 60}}
0.818044 (0.003159) with: {'class_weight': {0: 100, 1: 70}}
0.826204 (0.002880) with: {'class_weight': {0: 100, 1: 80}}
0.831268 (0.002619) with: {'class_weight': {0: 100, 1: 90}}
0.834770 (0.002409) with: {'class_weight': {0: 100, 1: 100}}


In [12]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.974367 using {'class_weight': {0: 100, 1: 100}}
0.957892 (0.002003) with: {'class_weight': {0: 100, 1: 10}}
0.965399 (0.001561) with: {'class_weight': {0: 100, 1: 20}}
0.968848 (0.001524) with: {'class_weight': {0: 100, 1: 30}}
0.970745 (0.001343) with: {'class_weight': {0: 100, 1: 40}}
0.971894 (0.001250) with: {'class_weight': {0: 100, 1: 50}}
0.972762 (0.001217) with: {'class_weight': {0: 100, 1: 60}}
0.973319 (0.001105) with: {'class_weight': {0: 100, 1: 70}}
0.973818 (0.001057) with: {'class_weight': {0: 100, 1: 80}}
0.974147 (0.001036) with: {'class_weight': {0: 100, 1: 90}}
0.974367 (0.001009) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.916882 using {'class_weight': {0: 90, 1: 100}}
0.821332 (0.002933) with: {'class_weight': {0: 10, 1: 100}}
0.871599 (0.002992) with: {'class_weight': {0: 20, 1: 100}}
0.892810 (0.001996) with: {'class_weight': {0: 30, 1: 100}}
0.903750 (0.001707) with: {'class_weight': {0: 40, 1: 100}}
0.910147 (0.001750) with: {'class_weight': {0: 50, 1: 100}}
0.913860 (0.001749) with: {'class_weight': {0: 60, 1: 100}}
0.915906 (0.001857) with: {'class_weight': {0: 70, 1: 100}}
0.916844 (0.002300) with: {'class_weight': {0: 80, 1: 100}}
0.916882 (0.002259) with: {'class_weight': {0: 90, 1: 100}}
0.916748 (0.002102) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-insurance-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-insurance-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder